In [1]:

import numpy as np
import random as rd
import scipy.sparse as sp
from time import time


In [2]:
import numpy as np 
import pandas as pd 
import scipy.sparse as sp

import torch.utils.data as data

# import config

In [3]:
def load_all_custom(test_num=100):
	""" We load all the three file here to save time in each epoch. """
    
	'''train_data = pd.read_csv(
		'./data/train_df', header=None, names=['user', 'item'], 
		usecols=[0, 1], dtype={0: np.int32, 1: np.int32}) '''
	#train_data = pd.read_csv('../data/train_df')    
	train_data = pd.read_csv('./train_df')
	train_data = train_data[['uid', 'sid']]
	train_data['uid'] = train_data['uid'].apply(lambda x : int(x))
	train_data['sid'] = train_data['sid'].apply(lambda x : int(x))    
	train_data.columns = ['user', 'item']
    
	user_num = train_data['user'].max() + 1
	item_num = train_data['item'].max() + 1

	train_data = train_data.values.tolist()

	# load ratings as a dok matrix
	train_mat = sp.dok_matrix((user_num, item_num), dtype=np.float32)
	for x in train_data:
		train_mat[x[0], x[1]] = 1.0
        
	return train_mat

In [4]:
R = load_all_custom()

In [5]:
R

<6040x3260 sparse matrix of type '<class 'numpy.float32'>'
	with 599123 stored elements in Dictionary Of Keys format>

In [16]:
R

<6040x3260 sparse matrix of type '<class 'numpy.float32'>'
	with 599123 stored elements in Dictionary Of Keys format>

In [6]:
    def create_adj_mat(user_num, item_num, R):
        from time import time
        t1 = time()
        adj_mat = sp.dok_matrix((user_num + item_num, user_num + item_num), dtype=np.float32)
        adj_mat = adj_mat.tolil()
        R = R.tolil()

        adj_mat[:user_num, user_num:] = R
        adj_mat[user_num:, :user_num] = R.T
        adj_mat = adj_mat.todok()
        print('already create adjacency matrix', adj_mat.shape, time() - t1)

        t2 = time()

        def mean_adj_single(adj):
            # D^-1 * A
            rowsum = np.array(adj.sum(1))

            d_inv = np.power(rowsum, -1).flatten()
            d_inv[np.isinf(d_inv)] = 0.
            d_mat_inv = sp.diags(d_inv)

            norm_adj = d_mat_inv.dot(adj)
            # norm_adj = adj.dot(d_mat_inv)
            print('generate single-normalized adjacency matrix.')
            return norm_adj.tocoo()

        def normalized_adj_single(adj):
            # D^-1/2 * A * D^-1/2
            rowsum = np.array(adj.sum(1))

            d_inv_sqrt = np.power(rowsum, -0.5).flatten()
            d_inv_sqrt[np.isinf(d_inv_sqrt)] = 0.
            d_mat_inv_sqrt = sp.diags(d_inv_sqrt)

            # bi_lap = adj.dot(d_mat_inv_sqrt).transpose().dot(d_mat_inv_sqrt)
            bi_lap = d_mat_inv_sqrt.dot(adj).dot(d_mat_inv_sqrt)
            return bi_lap.tocoo()

        def check_adj_if_equal(adj):
            dense_A = np.array(adj.todense())
            degree = np.sum(dense_A, axis=1, keepdims=False)

            temp = np.dot(np.diag(np.power(degree, -1)), dense_A)
            print('check normalized adjacency matrix whether equal to this laplacian matrix.')
            return temp

        norm_adj_mat = mean_adj_single(adj_mat + sp.eye(adj_mat.shape[0]))
        # norm_adj_mat = normalized_adj_single(adj_mat + sp.eye(adj_mat.shape[0]))
        mean_adj_mat = mean_adj_single(adj_mat)

        print('already normalize adjacency matrix', time() - t2)
        return adj_mat.tocsr(), norm_adj_mat.tocsr(), mean_adj_mat.tocsr()

In [7]:
train_df = pd.read_csv('./train_df')

In [8]:
total_df = pd.read_csv('./total_df')

In [9]:
u_num = len(total_df.uid.unique())
s_num = len(total_df.sid.unique())

In [10]:
print(u_num)
print(s_num)

6040
3260


In [11]:
adj_mat, norm_adj_mat, mean_adj_mat = create_adj_mat(u_num, s_num, R)

already create adjacency matrix (9300, 9300) 2.440239191055298
generate single-normalized adjacency matrix.
generate single-normalized adjacency matrix.
already normalize adjacency matrix 0.8638041019439697


In [12]:
sp.save_npz('s_adj_mat.npz', adj_mat)
sp.save_npz('s_norm_adj_mat.npz', norm_adj_mat)
sp.save_npz('s_mean_adj_mat.npz', mean_adj_mat)
